In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import Model
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3,preprocess_input

In [3]:
datagen = image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=preprocess_input
)

In [5]:
PATH = "dataset/"
train_generator = datagen.flow_from_directory(
    PATH,
    target_size=(224,224),
    class_mode="binary",
    subset="training",
    color_mode="rgb",
    batch_size=128,
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    PATH,
    target_size=(224,224),
    class_mode="binary",
    subset="validation",
    color_mode="rgb",
    batch_size=128,
    shuffle=True
)
## removing validation for final training

Found 11047 images belonging to 2 classes.
Found 2761 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'COVID': 0, 'Normal': 1}

In [6]:
11047+2761

13808

In [8]:
model = InceptionV3(include_top=False,weights="imagenet",input_shape=(224,224,3))

In [9]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [10]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation="relu")(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(1,activation="sigmoid")(d1)

model_new = Model(inputs=model.input,outputs=fc2)
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [11]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.00003)

In [12]:
for ix in range(len(model_new.layers)):
    print(ix,":",model_new.layers[ix])

0 : <keras.engine.input_layer.InputLayer object at 0x00000184F827EB50>
1 : <keras.layers.convolutional.Conv2D object at 0x00000184F8387100>
2 : <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000184F83873D0>
3 : <keras.layers.core.Activation object at 0x00000184F83FFB20>
4 : <keras.layers.convolutional.Conv2D object at 0x00000184F8399E80>
5 : <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000018488F334F0>
6 : <keras.layers.core.Activation object at 0x0000018488F15190>
7 : <keras.layers.convolutional.Conv2D object at 0x0000018488F33C40>
8 : <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000018488F3A8E0>
9 : <keras.layers.core.Activation object at 0x0000018488F40190>
10 : <keras.layers.pooling.MaxPooling2D object at 0x0000018488F40430>
11 : <keras.layers.convolutional.Conv2D object at 0x0000018488F3A850>
12 : <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000

In [15]:
from keras.callbacks import ModelCheckpoint

In [13]:
for ix in range(289):
    model_new.layers[ix].trainable = False

model_new.compile(loss="binary_crossentropy",optimizer=adam,metrics=["accuracy"])
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [16]:
callback = ModelCheckpoint("new_model/best.h5",monitor="val_accuracy",save_best_only=True)

In [17]:
hist = model_new.fit(train_generator,
                     epochs=20,
                     validation_data=validation_generator,
                     callbacks=[callback],
                     steps_per_epoch=80,
                     validation_steps=20
                    )

Epoch 1/20
80/80 [==============================] - 198s 2s/step - loss: 0.5067 - accuracy: 0.7447 - val_loss: 0.5088 - val_accuracy: 0.7391
Epoch 2/20
80/80 [==============================] - 143s 2s/step - loss: 0.4680 - accuracy: 0.7640 - val_loss: 0.4845 - val_accuracy: 0.7363
Epoch 3/20
80/80 [==============================] - 156s 2s/step - loss: 0.4474 - accuracy: 0.7813 - val_loss: 0.4860 - val_accuracy: 0.7367
Epoch 4/20
80/80 [==============================] - 158s 2s/step - loss: 0.4304 - accuracy: 0.7955 - val_loss: 0.4333 - val_accuracy: 0.7727
Epoch 5/20
80/80 [==============================] - 164s 2s/step - loss: 0.4236 - accuracy: 0.7995 - val_loss: 0.4773 - val_accuracy: 0.7480
Epoch 6/20
80/80 [==============================] - 157s 2s/step - loss: 0.4157 - accuracy: 0.8055 - val_loss: 0.4191 - val_accuracy: 0.7871
Epoch 7/20
80/80 [==============================] - 158s 2s/step - loss: 0.4017 - accuracy: 0.8197 - val_loss: 0.4343 - val_accuracy: 0.7656
Epoch 8/20
80

In [18]:
model_new.save_weights("new_model/best2.h5")

In [1]:
# model_new.to_json()

In [26]:
testimg = image.load_img("C:/Users/asus/Desktop/covid/Covid/positive.jpeg",target_size=(224,224))

In [27]:
testimg = image.img_to_array(testimg)

In [28]:
testimg=testimg/255.0

In [29]:
testimg = np.array([testimg])

In [30]:
testimg.shape

(1, 224, 224, 3)

In [35]:
np.round(model_new.predict(testimg))

array([[0.]], dtype=float32)

In [33]:
test_gen = image.ImageDataGenerator(rescale=1./255)

In [34]:
test_generator = test_gen.flow_from_directory("C:/Users/asus/Desktop/covid/",target_size=(224,224))

Found 2 images belonging to 2 classes.


In [36]:
model_new.load_weights("best.h5")

In [21]:
model_new.evaluate(validation_generator)

1/1 [==============================] - 4s 4s/step - loss: 0.4317 - accuracy: 0.7949


[0.43166521191596985, 0.7948718070983887]

In [53]:
model_new.save("new_model/model.h5")

In [39]:
model_new.load_weights("new_model/best.h5")

In [23]:
def pred_img(image_path):
    testimg = image.load_img(image_path,target_size=(224,224))
    testimg = image.img_to_array(testimg)
    testimg=testimg/255.0
    testimg = np.array([testimg])
    testimg = preprocess_input(testimg)
    pred = np.round(model_new.predict(testimg))
    if pred == 0:
        return "positive"
    return "negative"

In [34]:
pred_img("dataset/COVID/COVID-100.png")

'positive'

In [43]:
print(pred_img(f"dataset_backup/Normal/IM-0264-0001.jpeg"))

positive


In [70]:
from keras.models import load_model